# Clifford attractors


<img align='right' style='padding:30px 0px 40px 20px' 
src='./assets/screenshot.png'/>


[Clifford attractors](./attractors.py) are a type of iterative equation that traces the path of a particle through a 2D space using functions of sine and cosine terms that make interesting "attractor" patterns (covering only some portions of the possible space, in certain shapes). 

Here we use Numpy and Pandas to calculate a dataframe consisting of millions of such locations, using [Numba](https://numba.pydata.org) to make generating them 50X faster than bare Python. We'll then plot the results as a static image using [Datashader](http://datashader.org), which renders arbitrarily large data into fixed-sized images.

In [ ]:
import numpy as np
from numba import jit

@jit(parallel=True)
def clifford_trajectory(a, b, c, d, x0, y0, n):
    xs, ys = np.zeros(n), np.zeros(n)
    xs[0], ys[0] = x0, y0
    for i in np.arange(n-1):
        xs[i+1] = np.sin(a * ys[i]) + c * np.cos(a * xs[i])
        ys[i+1] = np.sin(b * xs[i]) + d * np.cos(b * ys[i])
    return xs, ys

We can visualize the resulting dataframe using [Datashader](http://datashader.org), with colormaps from [Colorcet](http://colorcet.pyviz.org):

In [ ]:
import datashader as ds, pandas as pd
from datashader import transfer_functions as tf
from colorcet import palette_n as ps

def clifford_plot(a=1.9, b=1.9, c=1.9, d=0.8, n=1000000, colormap=ps['kbc']):
    cvs = ds.Canvas(plot_width=600, plot_height=600)
    xs, ys = clifford_trajectory(a, b, c, d, 0, 0, n)
    agg = cvs.points(pd.DataFrame({'x':xs, 'y':ys}), 'x', 'y')
    return tf.shade(agg, cmap=colormap[::-1])

In [ ]:
clifford_plot(a=1.7, b=1.7, c=0.6, d=1.2, n=20000000, colormap=ps['dimgray'])

Twenty million data points from an attractor clearly makes an interesting shape! The shapes depend on the parameters provided, and we can now easily build a control panel for exploring the effect of those parameters, using interactive widgets from [Panel](http://panel.pyviz.org):

In [ ]:
import panel as pn
pn.extension()

widgets = {
    'a': pn.widgets.FloatSlider(value=1.9, end=2.0, step=0.1, name='a'),
    'b': pn.widgets.FloatSlider(value=1.9, end=2.0, step=0.1, name='b'),
    'c': pn.widgets.FloatSlider(value=1.9, end=2.0, step=0.1, name='c'),
    'd': pn.widgets.FloatSlider(value=0.8, end=1.0, step=0.1, name='d'),
    'n': pn.widgets.IntSlider(value=10000000, start=1000, end=20000000, step=100, name='n'),
    'colormap': pn.widgets.Select(value=ps['bmw'], options=ps, name='colormap'),
}

attr = pn.bind(clifford_plot, **widgets)
pn.Column(*widgets.values(), attr)

Here `bind()` enables you to build interactive components that respond to user inputs simply by binding widgets to functions. This lets us vary the arguments to the given function interactively by changing the widget values and see the results, as long as this notebook is backed by a live, running Python process.  

`bind()` is great for quick exploration, but it's not a dead end and we can use its components to build a more customized dashboard if we like.

We'll put the widgets into a column with a logo image and some Markdown text to explain how to use the dashboard, and put that column in a simple template with the output served in a dashboard:

In [ ]:
logo = "https://tinyurl.com/y9c2zn65/logo_stacked_s.png"

text = pn.panel("#### Use the widgets to vary the parameters of this "
                "[Clifford attractor](attractors.ipynb).\n\n"
                "#### Note that many values result in nearly "
                "blank plots that contain only a few scattered points."
                )

template = pn.template.BootstrapTemplate(
    header_background='#9944DA',
    title='Clifford Attractor',
    sidebar=[logo, text, *widgets.values()],
    main=[attr]
)
template.servable();

We could have displayed the new object here in the notebook by removing the semicolon at the end of the cell, but this expanded dashboard version isn't actually that useful in a notebook, where we already have ways of showing logos and text. Instead, the expanded version is meant for launching as a standalone server outside of the notebook, and so we've marked that object `.servable()` to declare that if someone later runs this notebook as a server process (using `panel serve --show CliffordInteract.ipynb`), your browser will open a separate window with the serveable object ready to explore or share, just like the screenshot at the top of this notebook.

So it's your choice -- use a static Image in a Jupyter cell, make it an interactive app using Panel's `bind`, or very easily reconfigure it into a shareable, deployable dashboard!